In [1]:
#Imports and libraries 
import numpy as np
import pandas as pd
import math
import csv
import re

In [2]:
#Loading Training Data
path = "trg.csv"
#Pandas
RawData = pd.read_csv("trg.csv",index_col=0)
RawData.head(n=10)

#Not Pandas
with open("trg.csv") as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    data = list(readCSV)
    headers = data[0]
    data = data[1:]
    data = np.array(data)


In [3]:
#Preprocessing - PANDAS
#Format - Uppercase letters
RawData['abstract'] = RawData['abstract'].str.upper()


#Remove negavite numbers and special characters (')
RawData['abstract'] = RawData['abstract'].str.replace(r"\s(-*\d*)*(-*\d*)*$\s|(-$)'",'',regex=True)
RawData['abstract'] = RawData['abstract'].str.replace(r"\s(-*\d*)*\s",'',regex=True)
RawData['abstract'] = RawData['abstract'].str.replace(r"\s(-?\d+)\s",'',regex=True)
RawData['abstract'] = RawData['abstract'].str.replace("'",'',regex=False)

#Split abstract string
RawData['abstract'] = RawData['abstract'].str.split(' ')

    
#Remove common words and sort the array
#Function to remove common words
def removeCommon(array):
    remove = ['THE','OF','A','AN','FOR','THAT','WITH','BY','AND',
          'OR','IS','ARE','BY','WAS','WERE','IT','ITS', 'TO',
             'WHICH', 'IN', 'HAVE', 'HAS', 'NO','NOT','AS', 'ALSO']
    #Loop to remove common words & numeric values in the abstract
    control=0
    while control<len(array):
        if array[control] in remove:
            array.pop(control)
        else:
            if array[control].isnumeric():
                array.pop(control)
            else:
                control += 1
    #Return the sorted array    
    array = array.sort()
    return array

#for word in remove:
#    control=1
#    while control<=len(array):
#        if array[control] in remove:
#            array.pop(word)
#    while len(array)>0 and array.pop()=word or control>len(array):
#        array.pop(word)
#        control++
        
#Apply function to the column data
RawData['abstract'].map(lambda x: removeCommon(x))

#Words Collector
# Identify different words in the abstracts 
# and count number of abstracts in which their appears 
def wordCollector(column):
    globalWords = {}
    for text in column:
        rowWords = {}
        for word in text:
            if not word in rowWords.keys():
                rowWords[word]=1;
        for new in rowWords.keys():
            if new in globalWords.keys():
                globalWords[new] += 1
            else:
                globalWords[new] = 1
    return globalWords

allWords = wordCollector(RawData['abstract'])
print("Total number of words:" + str(len(allWords)))

#Remove Outliers
def removeUnique(allWords, limit=1):
    words = list(allWords)
    for word in words:
        if allWords[word]<=limit:
            allWords.pop(word)
    return allWords       

allWords = removeUnique(allWords, 1)
print("Total number of words after remove unique :" + str(len(allWords)))

#Create and array with all words
words = list(allWords)
words.sort()

#Create classes array
classes = RawData['class'].unique()

#Calculate probability of each class in the given class column of the dataset
def classProbability(data):
    classProbs = {}
    classes = data.unique()
    for c in classes:
        classProbs[c] = data.where(data==c).count()/data.count()
    return classProbs
classprobs = classProbability(RawData['class'])
print(classprobs)


Total number of words:34095
Total number of words after remove unique :14348
{'B': 0.4005, 'A': 0.032, 'E': 0.536, 'V': 0.0315}


In [3]:
#Preprocessing - NO PANDAS
#Format - Uppercase letters
for row in data:
    row[2]=row[2].upper()
#    print(data[0:2])
    
#Remove negavite numbers and special characters (')
for row in data:
    row[2]=re.sub(r"\s(-*\d*)*\s",' ',row[2])
    row[2]=re.sub(r"[']",' ',row[2])

#Split abstract string
index = 0
abstracts = {}
for row in data:
    abstracts[index] = np.array(row[2].split(' '))
    index += 1
    
#Remove common words and sort the array
#Function to remove common words
def removeCommon(words):
#List of common words
    remove = ['THE','OF','A','AN','FOR','THAT','WITH','BY','AND',
          'OR','IS','ARE','BY','WAS','WERE','IT','ITS', 'TO',
             'WHICH', 'IN', 'HAVE', 'HAS', 'NO','NOT','AS', 'ALSO']
#Loop to remove common words & numeric values in the abstract
    control=0
    while control<len(words):
        if words[control] in remove:
            words[control]="-DELETE-"
        control += 1
    words = np.delete(words,np.argwhere(words=='-DELETE-')[:,0])
#Return the sorted array    
    words = np.sort(words)
    return words
       
#Apply function to the column data
for element in abstracts:
    abstracts[element] = removeCommon(abstracts[element])

#Words Collector
# Identify different words in the abstracts 
# and count number of abstracts in which their appears (frequency in data instances - NOT Individual)
def wordCollector(column):
    globalWords = {}
    for text in column:
        rowWords = {}
        for new in column[text]:
            if not new in rowWords.keys():
                rowWords[new]=1;
        for word in rowWords.keys():
            if word in globalWords.keys():
                globalWords[word] += 1
            else:
                globalWords[word] = 1
    return globalWords

allWords = wordCollector(abstracts)
print("Total number of words:" + str(len(allWords)))

#Remove Outliers
def removeUnique(allWords, limit=1):
    words = list(allWords)
    for word in words:
        if allWords[word]<=limit:
            allWords.pop(word)
    return allWords       

allWords = removeUnique(allWords, 1)
print("Total number of words after remove unique :" + str(len(allWords)))

#Create and array with all words
words = list(allWords)
words.sort()

#Create classes array
classes = np.unique(data[:,1])

#Calculate probability of each class in the given class column of the dataset
def classProbability(classes):
    classProbs = {}
    uniqueClass = np.unique(classes)
    for c in uniqueClass:
        classProbs[c] = np.count_nonzero(classes==c)/classes.size
    return classProbs
#classProbs = classProbability(data[:,1])
#print(classProbs)

def preprocessing(data):
    #Format - Uppercase letters
    for row in data:
        row[2]=row[2].upper()
        
    #Remove negavite numbers and special characters (')
    for row in data:
        row[2]=re.sub(r"\s(-*\d*)*\s",' ',row[2])
        row[2]=re.sub(r"[']",' ',row[2])
        
    #Split abstract string
    index = 0
    abstracts = {}
    for row in data:
        abstracts[index] = np.array(row[2].split(' '))
        index += 1
        
    #RemoveCommon words
    for element in abstracts:
        abstracts[element] = removeCommon(abstracts[element])
        
    return abstracts
#   
#    #Count frequency of words in Data
#    allWords = wordCollector(abstracts)
#    #words = list(allWords)
#    #words.sort()
#    print("Total number of words:" + str(len(allWords)))
#    
#    allWords = removeUnique(allWords, 1)
#    print("Total number of words after remove unique :" + str(len(allWords)))
#    
#    #Create and array with all words
#    words = list(allWords)
#    words = words.sort()
#    
#    #Create classes array
#    classes = np.unique(data[:,1])
#    
#    #Calculate classes probabilities
#    classProbs = classProbability(data[:,1])
#    #print(classProbs)


Total number of words:29810
Total number of words after remove unique :13191


In [75]:
#training = np.asarray(list(abstracts.items()))
#print(training[:2,1])
#training = np.asarray(list(abstracts.items())[:10])
#training = np.asarray(abstracts)

#for row in data:
#    row[2]=row[2].upper()

#print(data[0:2])

#classData = RawData.loc[RawData['class'] == 'E']
#classData.head(n=5)
#print(classData.size)

#allWords = wordCollector(RawData['abstract'])
#words = list(allWords)
#words.sort()
#print(words)
#print(allWords)
#RawData['abstract'].map(lambda x: removeCommon(x))
#RawData['abstract'].head(n=10)
#RawData.head(n=5)
#new = removeCommon(RawData['abstract'][1])
#print(new)

#removeCommon(RawData['abstract'][1])
#print(RawData['abstract'][1])

    #classPriorProbs = np.zeros(len(targets))
  #wordCount = len(dict.fromkeys(instances))

    #for c in classPriorProbs:
    #    c = np.zeros(len(words))

    
            #targetWords[target].pop('total')
        #countClassWords = len(targetWords[target].keys())
        
        #        for prob in classPriorProbs[x]:   
#            postProb[x] =+ instance.count()*math.log(classPriorProbs[x][prob])
#        print(postProb)  
#        print(classCount, classTotal)

        #countClassWords = len(targetWords[target].keys())
    
    #training = np.asarray(list(abstracts.items())[:10])
#training = np.asarray(abstracts)

#print(training.loc[training['class']=='A']['abstract'])
#print(classWords)
#print(classWords.keys())
#print(training['abstract'][1])



#classData = RawData.loc[RawData['class'] == 'E']
#training.head(n=10)
#classData.head(n=5)

#print(priorProbs)
#print(training.loc[training['class']=='A']['abstract'])
#print(classWords)
#print(classWords.keys())
#print(training['abstract'][1])


#    testFold = DataTraining[cycle]
#    trainingFolds = DataTraining.pop(cycle)
#    performance[cycle] = runtest(trainingFolds, testFold)
#    cycle++ 
#cycle = 0
#averagePerformance = np.sum(performance)/folds

In [5]:
#Naive Bayes Model - PANDAS

#General Functions

#Calculate count of word for a given class
def classWords(target, instances):
    classWords = {}
    classWords['total']=0
    for text in instances:
        for word in text:
            if not word in classWords.keys():
                classWords[word] = 1
            else:
                classWords[word] += 1
            classWords['total']+=1
    return classWords

#Caculate the prior probabilities of an instances for all classes
def rowProb(words, instances, targets, targetWords):
    classPriorProbs = {}
    #Data Vocabulary (Count of unique words)
    wordCount = len(words)
    abstractWords = dict.fromkeys(instances)
# Evaluation per class
    for target in targets:        
        classPriorProbs[target] = {}
        totalClassWords = targetWords[target]['total']
#Evaluation of each word in the instances        
        for word in abstractWords:
            if not word in targetWords[target].keys():
                classPriorProbs[target][word] = 1 / (totalClassWords + wordCount)
            else:
                classPriorProbs[target][word] = \
                (targetWords[target][word] + 1 ) / (totalClassWords + wordCount)
    return classPriorProbs

def postProbBasic():
    return False

#Calculate 
def postProb(instance, classPriorProbs,classColumn):
    postProb = {}
    words = {}
# Count unique word of an instance
    for word in dict.fromkeys(instance):
        words[word] = instance.count(word)
# Count unique words in a class
    for x in classColumn.unique():
        classCount = classColumn.where(classColumn==x).count()
        classTotal = classColumn.count()
        postProb[x] =+ math.log(classCount/classTotal)
# Calculate posterior probability of instance
        for word in words:
            postProb[x] += words[word]*math.log(classPriorProbs[x][word])
 
    return postProb

def classCode(classes,postProb):
    prediction = {}
    pre = min(postProb, key=postProb.get)
    return prediction


#Example
#Select training
training = RawData.loc[0:10]
#Determine classes
classes = RawData['class'].unique()
#Get words count of instances of each class value
classesWords = {}
for x in classes:
    classesWords[x] = classWords(x,training.loc[training['class']==x]['abstract'])

priorProbs = rowProb(words,training['abstract'][1],classes,classesWords)

classifier = postProb(training['abstract'][1], priorProbs,training['class'])
print(classifier)
max(classifier.keys(), key=(lambda k: classifier[k]))


    

{'B': -12.97336932169116, 'A': -16.184723521634215, 'E': -15.429354947601855, 'V': -19.025625059801197}


'B'

In [4]:
#Naive Bayes Model - NO PANDAS

#General Functions

#Calculate count of word of instances for a given class
#target = class to count words
#data = dataset
#instances = index of instances of the target class
def classWords(target, data, instances):
    cWords = {}
    cWords['total']=0
    for row in instances:
        for word in data[row]:
            if not word in cWords.keys():
                cWords[word] = 1
            else:
                cWords[word] += 1
            cWords['total']+=1
    return cWords

#Calculate the prior probabilities of an instances for all classe
#words = Vocabulary of the dataset
#instance = instances | row evaluated
#targets = predictable classes
#targetwords = count of word of all instances of the same class
def conditionalProb(words, instances, targets, targetWords):
    classPriorProbs = {}
#Data Vocabulary (Count of unique words)
    wordCount = len(words)
    abstractWords = np.unique(instances)
# Evaluation per class
    for target in targets:        
        classPriorProbs[target] = {}
        totalClassWords = targetWords[target]['total']
        temp = targetWords[target].pop('total')

#Evaluation of each word in the instances        
        for word in abstractWords:
            if not word in targetWords[target].keys():
                classPriorProbs[target][word] = 1 / (totalClassWords + wordCount)
            else:
                classPriorProbs[target][word] = \
                (targetWords[target][word] + 1 ) / (totalClassWords + wordCount)
        targetWords[target]['total']=temp
    return classPriorProbs

def postProbBasic():
    return False

#Classifier - Calculate class probability of and instance
#instance = instance evaluated
#condProb = conditional probabilities - classPriorProbs - prob of word for each class
#priorProb = Prior probabilities for each class
def classProb(instance, condProbs, priorProb):
    postProb = {}
    words = {}
# Count unique word of an instance
    for word in np.unique(instance[1]):        
        words[word] = np.count_nonzero(instance[1]==word)
# Count unique words in a class
    for x in priorProb.keys():
        postProb[x] = math.log(priorProb[x])
# Calculate posterior probability of instance
        for word in words:
            postProb[x] += words[word]*math.log(condProbs[x][word])   
    return postProb

def classCode(classes,postProb):
    prediction = {}
    pre = min(postProb, key=postProb.get)
    return prediction

#Performances - Asses acurracy of model
def modelPerformance(predictions, real):
    valid=0;
    total = len(predictions)
    for prediction in range(len(predictions)):
        if predictions[prediction]==real[prediction][1]:
            valid += 1
    performance=valid/total
    return performance
   

In [131]:
#Example
#Select training
training = list(abstracts.items())[:1000]

#Determine classes
classes = np.unique(data[:,1])

#Determine Instances
numInstances = len(training)

#Get words count of instances of each class value
classesWords = {}
for c in classes:
    classesWords[c] = classWords(c, training, np.where(data[:numInstances]==c)[0])

prediction =[]
for elem in training:
    
#Get conditional probabilities of rows
    condProbs = conditionalProb(words,training[elem[0]],classes,classesWords)

#Classifier - Selecting a class
    classifier = classProb(training[elem[0]], condProbs, classProbs)
    #print(classifier)
    prediction.append(max(classifier.keys(), key=(lambda k: classifier[k])))
#print(np.array(prediction))
#print(data[:numInstances,1])


TypeError: unhashable type: 'numpy.ndarray'

In [5]:
#Cross-Validation Test
folds = 10
samplesData = {}
cycle = 0
performance = np.zeros(folds)

#Split data 
#Creating Folds
foldSize= int(len(data)/folds)
initialSample = 0
finalSample = foldSize
for fold in range(folds):
    if fold == folds:
        samplesData[fold] = data[initialSample:,0:3]
    else:
        samplesData[fold] = data[initialSample:finalSample,0:3]
        initialSample = finalSample
        finalSample += foldSize

print(samplesData[1])
#Evaluate model performances with cross-validation
#for(cycle; cycle<folds; cycle++)
#training = {}
#for sample in samples:
#    print(sample)
#    test = samples[sample]
#    for trainingSample in samples:
#        print(trainingSample)
#        if sample!=trainingSample:
#            print(samples[sample])
#            training = training.update(samples[sample])
#    print(len(training))
#    testFold = DataTraining[cycle]
#    trainingFolds = DataTraining.pop(cycle)
#    performance[cycle] = runtest(trainingFolds, testFold)
#    cycle++ 
#cycle = 0
#averagePerformance = np.sum(performance)/folds
    

[['401' 'B'
  'HELICOBACTER PYLORI ONE OF THE MOST COMMON BACTERIAL PATHOGENS OF HUMANS COLONIZES THE GASTRIC MUCOSA WHERE IT APPEARS TO PERSIST THROUGHOUT THE HOST S LIFE UNLESS THE PATIENT IS TREATED COLONIZATION INDUCES CHRONIC GASTRIC INFLAMMATION WHICH CAN PROGRESS TO A VARIETY OF DISEASES RANGING IN SEVERITY FROM SUPERFICIAL GASTRITIS AND PEPTIC ULCER TO GASTRIC CANCER AND MUCOSAL-ASSOCIATED LYMPHOMA STRAIN-SPECIFIC GENETIC DIVERSITY HAS BEEN PROPOSED TO BE INVOLVED IN THE ORGANISM S ABILITY TO CAUSE DIFFERENT DISEASES OR EVEN BE BENEFICIAL TO THE INFECTED HOST AND TO PARTICIPATE IN THE LIFELONG CHRONICITY OF INFECTION HERE WE COMPARE THE COMPLETE GENOMIC SEQUENCES OF TWO UNRELATED H PYLORI ISOLATES THIS IS TO OUR KNOWLEDGE THE FIRST SUCH GENOMIC COMPARISON H PYLORI WAS BELIEVED TO EXHIBIT A LARGE DEGREE OF GENOMIC AND ALLELIC DIVERSITY BUT WE FIND THAT THE OVERALL GENOMIC ORGANIZATION GENE ORDER AND PREDICTED PROTEOMES SETS OF PROTEINS ENCODED BY THE GENOMES OF THE TWO STRAINS A

In [12]:
#Cross-Validation Test
folds = 10
samplesData = {}
cycle = 0
performance = np.zeros(folds)

#Split data 
#Creating Folds
foldSize= int(len(data)/folds)
initialSample = 0
finalSample = foldSize
for fold in range(folds):
    if fold == folds:
        samplesData[fold] = data[initialSample:,0:3]
    else:
        samplesData[fold] = data[initialSample:finalSample,0:3]
        initialSample = finalSample
        finalSample += foldSize
        
#Evaluate model performances with cross-validation
for sample in range(folds):
    trainingData = []   # id, class
    print("Fold: " + str(sample))
    testData = samplesData[sample]     #id, class, abstracts
    for trainingSample in samplesData:        
        if sample!=trainingSample:
            for x in range(len(samplesData[trainingSample])):
                trainingData.append(samplesData[trainingSample][x])

    trainingData = np.array(trainingData)
    
    #Determine Instances
    numInstances = len(trainingData)
    print("Fold lenght:" + str(numInstances))
    
    #Preprocessing training Data
    training = preprocessing(trainingData)
    
    #Select training
    #training = abstracts
    
    #Determine classes
    classes = np.unique(trainingData[:,1])
    
    #Determine class probabilities
    classProbs = classProbability(trainingData[:,1])
    
    #Calculate words frequency
    allWords = wordCollector(training)
    print("Total number of words:" + str(len(allWords)))
    
    allWords = removeUnique(allWords, 1)
    print("Total number of words after remove unique :" + str(len(allWords)))
    
    #Create and array with all words
    words = list(allWords)
    words.sort()
    
    #Get words count of instances of each class value
    classesWords = {}
    for c in classes:
        classesWords[c] = classWords(c, training, np.where(trainingData==c)[0])
    
    predictions =[]
    for elem in range(len(testData)):
        
        #Get conditional probabilities of rows
        condProbs = conditionalProb(words,testData[elem],classes,classesWords)
        
        #Classifier - Selecting a class
        classifier = classProb(testData[elem], condProbs, classProbs)
        #Select final classification
        predictions.append(max(classifier.keys(), key=(lambda k: classifier[k])))
    
    #Measure fold performance
    performance[sample]= modelPerformance(predictions,testData)
    averagePerformance = sum(performance)/len(performance)
    print(performance)
    print(averagePerformance)


    

Fold: 0
Fold lenght:3600
Total number of words:27651
Total number of words after remove unique :12358
[0.395 0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
0.0395
Fold: 1
Fold lenght:3600
Total number of words:27683
Total number of words after remove unique :12279
[0.395 0.41  0.    0.    0.    0.    0.    0.    0.    0.   ]
0.08049999999999999
Fold: 2
Fold lenght:3600
Total number of words:27590
Total number of words after remove unique :12183
[0.395 0.41  0.375 0.    0.    0.    0.    0.    0.    0.   ]
0.118
Fold: 3
Fold lenght:3600
Total number of words:27732
Total number of words after remove unique :12299
[0.395 0.41  0.375 0.41  0.    0.    0.    0.    0.    0.   ]
0.15899999999999997
Fold: 4
Fold lenght:3600
Total number of words:27770
Total number of words after remove unique :12240
[0.395 0.41  0.375 0.41  0.43  0.    0.    0.    0.    0.   ]
0.202
Fold: 5
Fold lenght:3600
Total number of words:27932
Total number of words after remove unique :12321
[0.395  0.41   0.37

In [190]:
def modelPerformance(predictions, real):
    valid=0;
    total = len(predictions)
    for prediction in range(len(predictions)):
        if predictions[prediction]==real[prediction][1]:
            valid += 1
    performance=valid/total
    return performance

In [191]:
#Evaluate model performances with cross-validation
print(samplesData.keys())
for sample in range(folds):
    trainingData = []   # id, class
    trainingData = 0
    print("Fold: " + str(sample))
    testData = samplesData.pop(sample)     #id, class, abstracts
    for trainingSample in samplesData.keys():       
        print(trainingSample)
        if trainingSample<len(samples)-1 and trainingSample > 0:
            if trainingData==0:
                index1 = list(samplesData.keys())[trainingSample]
                index2 = list(samplesData.keys())[trainingSample-1]
                print(index1)
                trainingData = np.stack(samplesData[index1],samplesData[index2])
            else:
                trainingData = np.stack(trainingData,samplesData[trainingSample])


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Fold: 0
1
2


TypeError: only size-1 arrays can be converted to Python scalars

In [70]:
#Generate Solution and Export to CSV
RawData.to_cvs('amon897.csv')

AttributeError: 'DataFrame' object has no attribute 'to_cvs'